# Preprocessor Tuning (Ön İşlemci Ayarlama)

## (0) `tumors` Dataset

* 👩🏻‍⚕️ Aşağıdaki veri seti, <font color=red>malignant</font> veya <font color=green>benign</font> olan tümörleri tanımlar. 
* 🎯 Görev, mümkün olduğunca çok sayıda <font color=red>malignant</font> tümörü tespit etmektir.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

url = "https://d32aokrjazspmn.cloudfront.net/materials/tumors_dataset.csv"
data = pd.read_csv(url)

data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,malignant
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,NaN,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,NaN,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.2,1575.0,NaN,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [2]:
round(data.malignant.value_counts(normalize = True),2)

malignant
0    0.63
1    0.37
Name: proportion, dtype: float64

## (1) Pipeline Oluşturma

❓ **Soru: Bir Pipeline Oluşturma** ❓

Aşağıdaki adımları `pipeline` adlı bir **`Pipeline`** nesnesi içinde birleştirin:

1. Eksik değerleri **`KNNImputer`** ile doldurun  
2. Tüm (sayısal) feature’ları **`MinMaxScaler`** ile ölçekleyin  
3. Varsayılan parametrelerle bir **`LogisticRegression`** modeli kurun

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

pipeline=Pipeline(steps=[
    ('imputer',KNNImputer()),
    ('scaler',MinMaxScaler()),
    ('model',LogisticRegression())])

## (2) Pipeline modelini optimize etme

❓ **Soru (Bir Pipeline üzerinde GridSearch)** ❓

* KNN imputer için **en uygun komşu sayısı** hangisidir: **2, 5 veya 10**?
    * Pipeline’ınız üzerinde bir **GridSearch** gerçekleştirin ve cevabınızı `n_best` adlı değişkende saklayın.
    * _Dikkat: Grid Search sırasında görevle **alakalı bir scoring metriği** kullandığınızdan emin olun, sadece söylemiş olalım 🙂_
* Bu challenge’da bir train/test split kullanmak yerine, **tüm veri seti üzerinde GridSearch** yapmaktan çekinmeyin. Buradaki amaç sadece **Pipeline’lara alışmak** 🙂

In [13]:
n_best = 5

In [18]:
from sklearn.model_selection import GridSearchCV


gridCV=GridSearchCV(pipeline,
                   param_grid={'imputer__n_neighbors' : [2,5,10]},
                    cv=5,
                    scoring="accuracy")

In [19]:
gridCV.fit(data.drop(columns="malignant"), data['malignant'])
gridCV.best_params_

{'imputer__n_neighbors': 5}

## (3)  Pipeline'nın Değerlendirilmesi

❓ **Soru: En iyi pipeline’ın performansı nedir?** ❓

- En iyi pipeline’ınızı mutlaka **cross-validation** ile değerlendirdiğinizden emin olun!
- Sonucu `cv_score` adlı bir değişkende **float** bir sayı olarak saklayın

In [ ]:
cv_score = None

In [28]:
from sklearn.model_selection import cross_validate,cross_val_score
cv_score=cross_val_score(
    gridCV.best_estimator_,
    data.drop(columns="malignant"),
    data['malignant'],
    cv=5,
    scoring='recall').mean()
cv_score


0.9244739756367663

In [29]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'solution', 
    n_best = n_best,
    cv_score=cv_score
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/temel/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/temel/S17D1-S-Data-Preprocessor-Tuning/tests
plugins: anyio-4.8.0, dash-3.3.0, typeguard-4.4.2
collecting ... collected 2 items

test_solution.py::TestSolution::test_n_neighbours PASSED                 [ 50%]
test_solution.py::TestSolution::test_score_good_enough PASSED            [100%]

============================== 2 passed in 1.05s ===============================


💯 You can commit your code:

git add tests/solution.pickle

git commit -m 'Completed solution step'

git push origin master



## (4) Eğitilmiş ve pipeline içine alınmış bir model ile tahmin yapma

👇 İşte yeni bir tümör.

In [30]:
new_url = "https://d32aokrjazspmn.cloudfront.net/materials/new_tumor.csv"

new_data = pd.read_csv(new_url)
new_data

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.0186,0.0134,0.01389,0.003532,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.186,0.275,0.08902


❓ **Soru: En iyi (optimal) pipeline’ınızı kullanarak yeni tümörün malignant olup olmadığını tahmin edin** ❓

In [31]:
gridCV.best_estimator_.predict(new_data)

array([1])

In [32]:
gridCV.best_estimator_.predict_proba(new_data)

array([[0.03203546, 0.96796454]])

🏁 Tebrikler! Artık pipeline oluşturma konusunda uzmansınız!

💾 Notebook’unuzu `git add/commit/push` yapmayı unutmayın...
